In [1]:
import pandas as pd
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model
import torch
from trl import SFTTrainer, setup_chat_format



c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Importing the dataset
dataset = load_dataset("yunfan-y/trump-qa", split="train")

# Load the tokenizer and model
model_name = "google/gemma-2-2b-it"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
)

# Assuming setup_chat_format is a custom function you've defined elsewhere
# If not, you might need to implement or import it
model, tokenizer = setup_chat_format(model, tokenizer)
model = get_peft_model(model, peft_config)

def format_chat_template(row):    
    row_json = [{"role": "system", "content": row["instruction"]},
               {"role": "user", "content": row["input"]},
               {"role": "assistant", "content": row["output"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

# Apply the formatting to the dataset
dataset = dataset.map(
    format_chat_template
)

print(dataset)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]


Dataset({
    features: ['output', 'input', 'instruction', 'text'],
    num_rows: 33596
})


In [3]:
# Setting Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./result",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    group_by_length=True,
    report_to="wandb",
    weight_decay=0.01,
)
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= 512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

model.config.use_cache = False
trainer.train()

model.push_to_hub("yunfan-y/trump-gemma-qa")
trainer.push_to_hub("yunfan-y/trump-gemma-qa")

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\trl\trainer\sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packag

  0%|          | 1/16798 [00:00<2:55:22,  1.60it/s]

{'loss': 3.5246, 'grad_norm': 3.473902463912964, 'learning_rate': 2e-05, 'epoch': 0.0}


  0%|          | 2/16798 [00:00<1:52:32,  2.49it/s]

{'loss': 4.0154, 'grad_norm': 3.20951771736145, 'learning_rate': 4e-05, 'epoch': 0.0}


...

{'loss': 0.2894, 'grad_norm': 0.7569541931152344, 'learning_rate': 0.00011675005956635691, 'epoch': 0.42}


 42%|████▏     | 6999/16798 [32:04<49:27,  3.30it/s]

{'loss': 0.2828, 'grad_norm': 0.7802401781082153, 'learning_rate': 0.00011673814629497259, 'epoch': 0.42}


 42%|████▏     | 7000/16798 [32:05<51:45,  3.16it/s]c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\peft\utils\save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 0.5326, 'grad_norm': 1.1437932252883911, 'learning_rate': 0.00011672623302358829, 'epoch': 0.42}
